In [9]:
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


# モデルの定義
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)  # 入力層
        self.fc2 = nn.Linear(128, 10)  # 出力層

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # 28x28の画像を1次元に変換
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [10]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()  # 勾配をリセット
        output = model(data)
        loss = criterion(output, target)  # 損失の計算
        loss.backward()  # 勾配の計算
        optimizer.step()  # 重みの更新

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}] Loss: {loss.item():.6f}')


# テストデータでの評価
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # 損失の累積
            pred = output.argmax(dim=1, keepdim=True)  # 最も高い確率を持つクラスを予測
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(
        f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n')

In [11]:
# デバイスの設定 (GPUが利用可能であればGPUを使用)
import torch
import torch_directml

device = torch_directml.device()

# データの前処理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # MNISTの平均と標準偏差で正規化
])

# データセットのダウンロードとローダーの作成
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# モデルのインスタンスを作成
model = SimpleNN().to(device)

# 損失関数とオプティマイザの設定
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# エポックごとにトレーニングとテストを実行
for epoch in range(1, 6):
    train(model, device, train_loader, optimizer, criterion, epoch)
    test(model, device, test_loader, criterion)
    torch.cuda.empty_cache()

Train Epoch: 1 [0/60000] Loss: 2.352124
Train Epoch: 1 [3200/60000] Loss: 0.361284
Train Epoch: 1 [6400/60000] Loss: 0.414306
Train Epoch: 1 [9600/60000] Loss: 0.449147
Train Epoch: 1 [12800/60000] Loss: 0.200911
Train Epoch: 1 [16000/60000] Loss: 0.494891
Train Epoch: 1 [19200/60000] Loss: 0.178510
Train Epoch: 1 [22400/60000] Loss: 0.198683
Train Epoch: 1 [25600/60000] Loss: 0.062372
Train Epoch: 1 [28800/60000] Loss: 0.433395
Train Epoch: 1 [32000/60000] Loss: 0.250336
Train Epoch: 1 [35200/60000] Loss: 0.223149
Train Epoch: 1 [38400/60000] Loss: 0.085889
Train Epoch: 1 [41600/60000] Loss: 0.293438
Train Epoch: 1 [44800/60000] Loss: 0.112367
Train Epoch: 1 [48000/60000] Loss: 0.236079
Train Epoch: 1 [51200/60000] Loss: 0.267552
Train Epoch: 1 [54400/60000] Loss: 0.282885
Train Epoch: 1 [57600/60000] Loss: 0.154692

Test set: Average loss: 0.0037, Accuracy: 9631/10000 (96.31%)

Train Epoch: 2 [0/60000] Loss: 0.027536
Train Epoch: 2 [3200/60000] Loss: 0.019710
Train Epoch: 2 [6400/600